In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Criando variável para armazenar o link da Fake API.
sdw2023_api_url = 'https://loweringly-unmellifluent-yamileth.ngrok-free.dev'

#Extract

Extrair a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, foi feita uma requisição GET para obter os dados do usuário correspondente.

Utilização de Fake API local exposta na internet com a criação de um túnel público via ngrok.

In [8]:
#TODO Extrair os IDs do arquivo csv.
df = pd.read_csv('sdw2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1, 2, 3, 4, 5, 6]


In [9]:
#TODO Obter os dados de cada ID usando a API da Santander Dev Week 2023
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1,
    "name": "Eliza",
    "account": {
      "number": "0001-01",
      "agency": "2025",
      "balance": 624.12,
      "limit": 2000.0
    },
    "card": {
      "number": "xxxx xxxx xxxx 1111",
      "limit": 2000.0
    }
  },
  {
    "id": 2,
    "name": "Jo\u00e3o",
    "account": {
      "number": "0001-02",
      "agency": "2025",
      "balance": 624.13,
      "limit": 1500.0
    },
    "card": {
      "number": "xxxx xxxx xxxx 2222",
      "limit": 1500.0
    }
  },
  {
    "id": 3,
    "name": "Maria",
    "account": {
      "number": "0001-03",
      "agency": "2025",
      "balance": 624.14,
      "limit": 3000.0
    },
    "card": {
      "number": "xxxx xxxx xxxx 3333",
      "limit": 3000.0
    }
  },
  {
    "id": 4,
    "name": "Jo\u00e3o da Silva",
    "account": {
      "number": "00000000-0",
      "agency": "0001",
      "balance": 1320.64,
      "limit": 1000.0
    },
    "card": {
      "number": "xxxx xxxx xxxx 0000",
      "limit": 1000.0
    

#Transform

Utilização de API do Gemini para gerar uma mensagem de marketing personalizada para cada usuário.

In [10]:
#TODO Implementar a integração com o Google Gemini.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
from dotenv import load_dotenv
import os

load_dotenv('/content/drive/MyDrive/chave_api_gemini.env') #Deixei a chave oculta em um doc .env, para não ficar exposta no código.
api_key = os.getenv("GOOGLE_API_KEY")

from google import genai
from google.genai import types
client = genai.Client(api_key=api_key)

In [19]:
def generate_ai_news(user):
  completion = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        system_instruction="Você é um especialista de marketing que trabalha faz anos para uma agência bancária."
    ),
    contents=f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos. A mensagem deve ser feita levando em consideração o perfil de cliente baseado no seu saldo em conta de {user['account']['balance']} reais e seu limite do cartão de {user['card']['limit']} reais, porém, não fale de forma explícita o valor que o cliente tem ou insinuar que ele tem pouco ou muito dinheiro. A mensagem não deve ter mais de 100 caracteres."
  )
  return completion.text

for user in users:
  user.setdefault("news", [])
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

with open("users_with_news.json", "w", encoding="utf-8") as f:
    json.dump(users, f, indent=2, ensure_ascii=False)

Eliza, seu dinheiro pode trabalhar para você! Invista no seu futuro.
João, o futuro financeiro começa hoje. Dê o primeiro passo nos investimentos!
Maria, faça seu dinheiro trabalhar para você. Pequenos passos hoje, grandes sonhos amanhã. Invista!
João, seus sonhos merecem crescer! Invista seu dinheiro e construa um futuro mais próspero.
Paulo, construa o futuro que você deseja. Invista agora!
Carlos, invista no seu amanhã. Cada valor conta. Comece hoje!


#Load

Atualizar a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [20]:
#TODO Atualizar os usuários na API da Santander Dev Week 2023 com os dados enriquecidos.
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user["id"]}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Eliza updated? True!
User João updated? True!
User Maria updated? True!
User João da Silva updated? True!
User Paulo da Silva updated? True!
User Carlos da Silva updated? True!
